# 📩 SMS Spam Classifier with TensorFlow

This project builds a machine learning model that classifies SMS messages as either **"ham" (not spam)** or **"spam"** using TensorFlow and natural language processing.

You'll walk through the entire ML workflow, including:

- 📥 Loading and preparing real-world SMS data  
- 🧼 Preprocessing text with a `TextVectorization` layer  
- 🧠 Building and training a neural network for binary classification  
- 📈 Evaluating model accuracy with confusion matrix, ROC curve, and prediction confidence plots  
- 🌀 Visualizing message patterns with word clouds, t-SNE, and misclassification analysis

The final result is a function that can predict whether a given message is spam, including the model’s confidence score.

> “Congratulations! You've won a free iPhone!” — spam  
> “Are we still on for dinner?” — ham

Let’s classify some texts! 🚀


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
!ls

In [ ]:
!ls -l /content

In [ ]:
!ls -l /content/sample_data

In [ ]:
column_names = ['label', 'message']
train_data_df = pd.read_csv(train_file_path, sep='\t', header=None, names=column_names)
test_data_df = pd.read_csv(test_file_path, sep='\t', header=None, names=column_names)

print(train_data_df.head())
print(train_data_df.tail())
print(len(train_data_df))
print(test_data_df.head())
print(len(test_data_df))

## Data Visualizations

### Histograms

In [ ]:
import seaborn as sns

# Visualize label counts
sns.countplot(x='label', data=train_data_df)
plt.title('Label Distribution in Training Data')
plt.xlabel('Label (ham/spam)')
plt.ylabel('Count')
plt.show()

In [ ]:
# Message Length Distribution

# Add a 'length' column to training data
train_data_df['length'] = train_data_df['message'].apply(len)

# Plot message length distribution
plt.hist(train_data_df['length'], bins=40, color='skyblue', edgecolor='black')
plt.title('Distribution of Message Lengths')
plt.xlabel('Message Length')
plt.ylabel('Frequency')
plt.show()

### Boxplot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Boxplot comparing message lengths between ham and spam
plt.figure(figsize=(8, 6))
sns.boxplot(x='label', y='length', data=train_data_df, palette={'ham':'green', 'spam':'red'})

plt.title('Message Lengths by Label', fontsize=14)
plt.xlabel('Message Type', fontsize=12)
plt.ylabel('Number of Characters in Message', fontsize=12)

plt.text(-0.3, train_data_df['length'].max() * 0.95, "👈 Ham messages are usually shorter", color='green')
plt.text(0.7, train_data_df['length'].max() * 0.95, "Spam messages often longer 👉", color='red')

plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

### Word Clouds

In [ ]:
from wordcloud import WordCloud

# Combine all spam and ham text separately
spam_text = ' '.join(train_data_df[train_data_df['label'] == 'spam']['message'])
ham_text = ' '.join(train_data_df[train_data_df['label'] == 'ham']['message'])

# Generate word clouds with custom color maps
spam_wc = WordCloud(
    width=600,
    height=400,
    background_color='white',
    colormap='hot'  # 🔥 red/yellow tones for angry spam
).generate(spam_text)

ham_wc = WordCloud(
    width=600,
    height=400,
    background_color='white',
    colormap='Greens'  # 🌿 calm and friendly ham
).generate(ham_text)

# Show them side by side
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(ham_wc, interpolation='bilinear')
plt.title("Common Words in Ham")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(spam_wc, interpolation='bilinear')
plt.title("Common Words in Spam")
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Create the model

# 1. Prepare your TextVectorization layer
vectorizer = keras.layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=100
)

# 2. Get train and test text
train_texts = train_data_df['message'].values
test_texts = test_data_df['message'].values

# 3. Adapt the vectorizer to training texts
vectorizer.adapt(train_texts)

# 4. Build the model
model = keras.Sequential()
model.add(vectorizer)
model.add(keras.layers.Embedding(input_dim=10000, output_dim=16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid')) # sigmoid = output is probability

# 5. Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# 6. Convert labels from strings to integers
train_labels = train_data_df['label'].map({'ham': 0, 'spam': 1}).values
test_labels = test_data_df['label'].map({'ham': 0, 'spam': 1}).values

# 7. Fit the model
model.fit(train_texts, train_labels, epochs=12, validation_data=(test_texts, test_labels))

### Model Prediction Confidence Distribution

In [ ]:
# Predict on test set
pred_probs = model.predict(tf.constant(test_texts)).flatten()
true_labels = test_labels

# Plot histogram of prediction confidence
plt.hist(pred_probs[true_labels == 0], bins=30, alpha=0.7, label='ham', color='green')
plt.hist(pred_probs[true_labels == 1], bins=30, alpha=0.7, label='spam', color='red')
plt.title('Model Prediction Confidence')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.legend()
plt.show()

### Confusion Matrix Heatmap

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Convert probabilities to binary predictions
pred_classes = (pred_probs > 0.5).astype(int)

# Create confusion matrix
cm = confusion_matrix(true_labels, pred_classes)

# Plot heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['ham', 'spam'], yticklabels=['ham', 'spam'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # Convert to Tensor (batch of 1 string)
  pred_tensor = tf.constant([pred_text])

  prediction = model.predict(pred_tensor)[0][0]

  label = 'spam' if prediction > 0.5 else 'ham'

  return [prediction, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

# Receiver Operating Characteristic (ROC) Curve

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(true_labels, pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # baseline
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.show()


### Waterfall Plot of Individual Predictions (Top Confident vs Misclassifications)

In [ ]:
# Predict and collect useful info
import pandas as pd

results_df = pd.DataFrame({
    'text': test_texts,
    'true_label': test_labels,
    'pred_prob': pred_probs,
    'pred_class': (pred_probs > 0.5).astype(int)
})

# Add info columns
results_df['correct'] = results_df['true_label'] == results_df['pred_class']
results_df['confidence'] = np.abs(results_df['pred_prob'] - 0.5) * 2  # how far from 0.5

# Sort by confidence
results_df = results_df.sort_values('confidence', ascending=False).reset_index(drop=True)

# Plot
plt.figure(figsize=(12, 6))
colors = results_df['correct'].map({True: 'green', False: 'red'})
plt.bar(range(len(results_df)), results_df['confidence'], color=colors)
plt.title('Prediction Confidence per Message (Green = Correct, Red = Incorrect)')
plt.xlabel('Message Index (sorted by confidence)')
plt.ylabel('Confidence Score')
plt.tight_layout()
plt.show()


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
